In [3]:
from tables import *
h5file = open_file("/mnt/disk100/persist_home/meawoppl/export_test.h5", mode="r+")

In [7]:
h5file.root.readout.cols.epoch.create_index()

124317972

In [9]:
h5file.root.readout.cols.norad_id.create_index()

124317972